In [1]:
import random as rd
import math

In [2]:
class Course:
    def __init__(self, code, nlesson, teacher, capacity):
        self.code = code
        self.nlesson = nlesson
        self.teacher = teacher
        self.capacity = capacity
        
    def __str__(self):
        return f"{self.code}"

In [3]:
class Room:
    def __init__(self, code, capacity):
        self.code = code
        self.capacity = capacity
        
    def __str__(self):
        return f"{self.code}"

In [4]:
class Schedule:
    def __init__(self, timetable):
        self.timetable = timetable
        
    def fitness(self):
        error = 0
        check = {}
        for [day, lesson, room, course] in self.timetable:
            if course.capacity > room.capacity:
                error += 1
            if (day, lesson, room) in check.keys():
                error += 1
            else:
                check[(day, lesson, room)] = 1
            if (day, lesson, course.teacher) in check.keys():
                error += 1
            else:
                check[(day, lesson, course.teacher)] = 1
            if (day, lesson, course) in check.keys():
                error += 1
            else:
                check[(day, lesson, course)] = 1
        fitness = 1.0 / (error + 1)
        return fitness

In [5]:
ncourse = 30
nroom = 12
nneighbor = 25

In [6]:
def random_course():
    courses = []
    for i in range(ncourse):
        courses.append(
            Course(i, rd.randint(3, 6), rd.randint(1, 8), rd.randint(30, 40))
        )
    return courses

courses = random_course()

def random_room():
    rooms = []
    for i in range(nroom):
        rooms.append(
            Room(i, rd.randint(36, 45))
        )
    return rooms

rooms = random_room()

In [7]:
def init_schedule():
    timetable = []
    for course in courses:
        for _ in range(course.nlesson):
            element = [
                rd.randint(2, 6), 
                rd.randint(1, 12), 
                rooms[rd.randint(0, nroom - 1)], 
                course
            ]
            timetable.append(element)
    return Schedule(timetable)

In [8]:
def localsearch(schedule):
    neighbors = []
    idxs = rd.sample(range(len(schedule.timetable)), k=nneighbor)
    for idx in idxs:
        [day, lesson, room, course] = schedule.timetable[idx]
        one = [day + 1 if day < 6 else 2, lesson, room, course]
        neighbor = schedule
        neighbor.timetable[idx] = one
        neighbors.append(neighbor)
        [day, lesson, room, course] = schedule.timetable[idx]
        two = [day - 1 if day > 2 else 6, lesson, room, course]
        neighbor = schedule
        neighbor.timetable[idx] = two
        neighbors.append(neighbor)
        [day, lesson, room, course] = schedule.timetable[idx]
        three = [day, lesson + 1 if lesson < 12 else 1, room, course]
        neighbor = schedule
        neighbor.timetable[idx] = three
        neighbors.append(neighbor)
        [day, lesson, room, course] = schedule.timetable[idx]
        four = [day, lesson - 1 if lesson > 1 else 12, room, course]
        neighbor = schedule
        neighbor.timetable[idx] = four
        neighbors.append(neighbor)
        [day, lesson, room, course] = schedule.timetable[idx]
        five = [day, lesson, rooms[rd.randint(0, nroom - 1)], course]
        neighbor = schedule
        neighbor.timetable[idx] = five
        neighbors.append(neighbor)
    return neighbors

In [9]:
def metropolis(neighbors, s, t):
    neighbor = rd.choice(neighbors)
    if neighbor.fitness() >= s.fitness():
        return neighbor
    if rd.random() < math.exp((neighbor.fitness() - s.fitness()) / t):
        return neighbor
    return s

In [10]:
def stimulated_annealing():
    s = init_schedule()
    opt = s
    cnt = 10000
    for i in range(0, cnt):
        t = cnt / 100000
        neighbors = localsearch(s)
        s = metropolis(neighbors, s, t)
        if opt.fitness() < s.fitness():
            opt = s
    return opt

In [11]:
opt = stimulated_annealing()

In [12]:
1 / opt.fitness()

38.0